In [15]:
import pandapower as pp
import numpy as np

In [22]:
def ieee4bus(p):
    """
    p:(mw) 2.5-5
    
    b1 -- b4
    |      |
    b2 -- b3
    
    """
    net = pp.create_empty_network()
    
    bus1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
    bus2 = pp.create_bus(net, vn_kv=20., name="Bus 2")
    bus3 = pp.create_bus(net, vn_kv=20., name="Bus 3")
    bus4 = pp.create_bus(net, vn_kv=20., name="Bus 4")

    pp.create_gen(net, bus1, p_mw=2.5, vm_pu=1.0, slack=True, name="Generator Bus 1", marginal_cost=40, min_p_mw=0)
    pp.create_gen(net, bus2, p_mw=1, vm_pu=1.0, name="Generator Bus 2", marginal_cost=50, min_p_mw=0)
    pp.create_gen(net, bus3, p_mw=1.5, vm_pu=1.0, name="Generator Bus 3", marginal_cost=60, min_p_mw=0)

    pp.create_line_from_parameters(net, from_bus=bus1, to_bus=bus2, length_km=1, r_ohm_per_km=0, x_ohm_per_km=1.5, c_nf_per_km=0, max_i_ka=1,p_max_mv=1.5)
    pp.create_line_from_parameters(net, from_bus=bus1, to_bus=bus4, length_km=1, r_ohm_per_km=0, x_ohm_per_km=1.5, c_nf_per_km=0, max_i_ka=1,p_max_mv=1.5)
    pp.create_line_from_parameters(net, from_bus=bus2, to_bus=bus3, length_km=1, r_ohm_per_km=0, x_ohm_per_km=1.5, c_nf_per_km=0, max_i_ka=1,p_max_mv=1.5)
    pp.create_line_from_parameters(net, from_bus=bus3, to_bus=bus4, length_km=1, r_ohm_per_km=0, x_ohm_per_km=1.5, c_nf_per_km=0, max_i_ka=1,p_max_mv=1.5)

    pp.create_load(net, bus4, p_mw=p, q_mvar=0.0, name="Load at Bus 4")

   
    pp.runpp(net)
    

    gen_output = net.res_gen[['p_mw']]
    gen_output['Bus'] = net.gen['bus'].map({bus1: 'Bus 1', bus2: 'Bus 2', bus3: 'Bus 3'})
    
    marginal_costs = {bus1: 40, bus2: 50, bus3: 60}
    gen_output['Marginal Cost ($/MWh)'] = net.gen['bus'].map(marginal_costs)
    gen_output['Generation Cost ($)'] = gen_output['p_mw'] * gen_output['Marginal Cost ($/MWh)']
    print(gen_output)
    cost_sum = np.sum(np.array(gen_output["Generation Cost ($)"]))
    
    return cost_sum

In [27]:
cost_sum=ieee4bus(2.732)
print("Cost($)",cost_sum)

    p_mw    Bus  Marginal Cost ($/MWh)  Generation Cost ($)
0  0.232  Bus 1                     40                 9.28
1  1.000  Bus 2                     50                50.00
2  1.500  Bus 3                     60                90.00
Cost($) 149.28
